In [1]:
#coding=utf-8
import xlwings as xw
import re

# 打開活頁簿檔案
file_path = ('Han-Ggi.xlsx')
wb = xw.Book(file_path)

In [2]:
source_sheet = wb.sheets['ji-im-tui-chiau']

end_row = source_sheet.range('A' + str(source_sheet.cells.last_cell.row)).end('up').row
print(f'end_row = {end_row}')

end_row = 1074


In [3]:
sheet = wb.sheets['ji-im-tui-chiau']

# 轉換成十五音【聲母】
siann_bu_dict = {
    'ㄅ': 'p', 
    'ㄆ': 'ph', 
    'ㄇ': 'm', 
    'ㆠ': 'b', 
    'ㄉ': 't', 
    'ㄊ': 'th', 
    'ㄋ': 'n', 
    'ㄌ': 'l', 
    'ㄍ': 'k', 
    'ㄎ': 'kh', 
    'ㄏ': 'h', 
    'ㆣ': 'g', 
    'ㄗ': 'ch', 
    'ㄘ': 'chh', 
    'ㄙ': 's', 
    'ㆡ': 'j', 
    'ㄐ': 'ch', 
    'ㄑ': 'chh', 
    'ㄒ': 's', 
    'ㆢ': 'j', 
    'ㄇ': 'm', 
    'ㄣ': 'n', 
    'ㄚ': 'ng',  
}
siann_pattern = str.maketrans(siann_bu_dict)

# 轉換成十五音【韻母】
un_bu_dict = {
    'ㄚ': 'a', 
    'ㄧ': 'i', 
    'ㄨ': 'u', 
    'ㆤ': 'e', 
    'ㄛ': 'oo', 
    'ㄜ': 'o',
    'ㄞ': 'ai',
    'ㄠ': 'au', 
    '儿': 'o', 
    'ㆩ': 'ann', 
    'ㆪ': 'inn', 
    'ㆫ': 'unn', 
    'ㆥ': 'enn', 
    'ㆧ': 'oonn',         
    'ㄢ': 'an',
    'ㄤ': 'ang',
    'ㄣ': 'n',
    'ㄥ': 'ng', 
    'ㄇ': 'm',
    'ㆬ': 'm',
    'ㆭ': 'ng',
    'ㄅ': 'p', 
    'ㄉ': 't', 
    'ㄍ': 'k', 
    'ㄏ': 'h', 
}
un_pattern = str.maketrans(un_bu_dict)
    
row = 1
while row < end_row:
    # Read data from source_sheet 
    chu_im  = str(source_sheet.range('B' + str(row)).value)
    # 沒有注音，取下一個儲存格
    if chu_im.strip() == '':
        row += 1
        continue

    # 如果有聲母／韻母為濁音，則進行聲母／韻母符號更換
    chu_im_str = chu_im.replace('ㄝ', 'ㆤ') \
                .replace('ㄅ。', 'ㆠ') \
                .replace('ㄍ。', 'ㆣ') \
                .replace('ㄗ。', 'ㆡ') \
                .replace('ㄐ。', 'ㆢ') \
                .replace('ㄚ。', 'ㆩ') \
                .replace('ㄧ。', 'ㆪ') \
                .replace('ㄨ。', 'ㆫ') \
                .replace('ㆤ。', 'ㆥ') \
                .replace('ㄛ。', 'ㆧ')
    chu_im_chars = len(chu_im_str)
    
    # ===================================================
    # 處理「聲母」
    # ===================================================
    
    # 判斷：是否有聲母（若字串開頭為韻母，則無聲母）
    # pattern = re.compile(r'^[ㄚㆩㄧㆪㄨㆫㄝㆥㄛㆧㄜ儿]')
    pattern = re.compile(r'^[ㄚㆩㄧㆪㄨㆫㆤㆥㄛㆧㄜ儿ㄞㄠㄢㄤㄣㄥ]')
    siann_bu = ''
    un_bu = ''
    if not pattern.match(chu_im_str):
        siann_bu += chu_im_str[0] 
        un_bu = chu_im_str[1:chu_im_chars-1]
    else:
        un_bu = chu_im_str[0:chu_im_chars-1]
            
    # ===================================================
    # 處理「聲調」
    # ===================================================
    siann_tiau = chu_im_str[chu_im_chars-1]

    # 入聲韻尾使用「ㄅ、ㄉ、ㄍ、ㄏ」
    buffer = chu_im_str[chu_im_chars-2:]
    pattern2 = re.compile(r'[ㄅㄉㄍㄏ][-+]$')
    if pattern2.match(buffer):
        if siann_tiau == '-':
            siann_tiau_2 = 8
        elif siann_tiau == '+':
            siann_tiau_2 = 4
    else:
        if siann_tiau == '-':
            siann_tiau_2 = 1
        elif siann_tiau == '+':
            siann_tiau_2 = 7
        elif siann_tiau == 'ˇ':
            siann_tiau_2 = 3
        elif siann_tiau == 'ˊ':
            siann_tiau_2 = 5
        elif siann_tiau == 'ˋ':
            siann_tiau_2 = 2

    # 轉換成十五音
    siann = ''
    if siann_bu != '':
        siann += siann_bu_dict[siann_bu]
    
    un_bu_str = un_bu.replace('ㄛㄍ', 'ok') \
                    .replace('ㄛㄥ', 'ong') \
                    .replace('ㆤㄣ', 'an') \
                    .replace('ㄧㄛㄍ', 'iok') \
                    .replace('ㄧㆤㄉ', 'iat') \
                    .replace('ㄧㆤㄍ', 'ek') \
                    .replace('ㄧㆤㄥ', 'eng') \
                    .replace('ㄨㄚ', 'oa') \
                    .replace('ㄨㄚㄉ', 'oat') \
                    .replace('ㄨㆤ', 'oe') \
                    .replace('ㄨㄢ', 'oan')  

    # un = un_bu.translate(siann_pattern).translate(un_pattern)
    un = un_bu_str.translate(un_pattern)
    
    # Write to target cell
    sheet.range('C' + str(row)).value = siann_bu
    sheet.range('D' + str(row)).value = un_bu
    sheet.range('E' + str(row)).value = siann_tiau
    sheet.range('F' + str(row)).value = siann
    sheet.range('G' + str(row)).value = un
    sheet.range('H' + str(row)).value = siann_tiau_2
    row += 1
  

KeyError: 'N'